In [1]:
# import os
# if os.path.basename(os.getcwd()) == 'notebooks':
#     os.chdir('..')

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from unidecode import unidecode

nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

def remove_colunas_dominantes(df: pd.DataFrame, limite_dominancia: float = 0.9) -> set:
    colunas_remover = set()
    for col in df.columns:
        valor_mais_frequente = df[col].value_counts(normalize=True, dropna=True).max()
        if valor_mais_frequente > limite_dominancia:
            colunas_remover.add(col)
    return colunas_remover


def preprocess_data(path: str, index_col: str, cols_normalize: list=None, explode_col: str=None) -> pd.DataFrame:
    df = pd.read_json(path).T.reset_index(drop=False).rename(columns={'index': index_col})
    
    if cols_normalize:
        for col in cols_normalize:
            df[pd.json_normalize(df[col]).columns] = pd.json_normalize(df[col])
            df = df.drop(columns=col)

    if explode_col:
        df = df.explode(column=explode_col)
        df = pd.concat([df.reset_index(drop=True), pd.json_normalize(df[explode_col]).reset_index(drop=True)], axis=1)
        df.drop(columns=explode_col, inplace=True)
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].replace('', pd.NA)
	# Remove colunas que não possuem dados relevantes, onde mais de 90% dos valores são NaN
    df.dropna(thresh=len(df) * 0.1, axis=1, inplace=True)
    # Remove colunas com valores dominantes
    colunas_remover = remove_colunas_dominantes(df, limite_dominancia=0.9)
    df.drop(columns=colunas_remover, inplace=True)
    return df

def limpar_texto(texto):
    if isinstance(texto, str):
        texto = unidecode(texto)  # Remove acentos
        texto = texto.lower()
        texto = texto.replace('/', ' ')
        texto = re.sub(r'[^\w\s]', '', texto)
        palavras = texto.split()
        palavras = [palavra for palavra in palavras if palavra not in stop_words]
        texto_limpo = ' '.join(palavras).strip()
        texto_limpo = re.sub(r'\s+', ' ', texto_limpo)
        return texto_limpo
    return None

def limpar_numeros_strings(valor):
    if isinstance(valor, str):
        return valor.replace('0', '').strip()
    return valor

def limpar_datas(valor):
    if isinstance(valor, str):
        valor = valor.strip()  
        if valor in ('0000-00-00', '0000', '0'):  
            return None
    try:
        data = pd.to_datetime(valor, errors='coerce', format='mixed')
        if data is pd.NaT:  
            return None
        if data.year < 1930 or data.year > 2030:  
            return None
        return data
    except:
        return None


def limpar_anos(valor):  
    if isinstance(valor, str):
        valor = valor.strip()  
        if valor in ('0000', '0'): 
            return None
    try:
        ano = int(valor)
        if 1900 <= ano <= 2025:
            return ano
        return None
    except:
        return None
        
# df_applicants
path_applicants = '../Datathon Decision/applicants.json'
cols_applicants = ['infos_basicas', 'informacoes_pessoais', 'informacoes_profissionais', 'formacao_e_idiomas']
df_applicants = preprocess_data(path=path_applicants, index_col='cod_applicant', cols_normalize=cols_applicants)

# df_prospects
path_prospects = '../Datathon Decision/prospects.json'
df_prospects = preprocess_data(path=path_prospects, index_col='cod_vaga', explode_col='prospects')

# df_vagas
path_vagas = '../Datathon Decision/vagas.json'
cols_vagas = ['informacoes_basicas', 'perfil_vaga', 'beneficios']
df_vagas = preprocess_data(path=path_vagas, index_col='cod_vaga', cols_normalize=cols_vagas)

[nltk_data] Downloading package stopwords to /home/felip/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## --- TRATANDO COLUNAS TABELA APPLICANTS ---

colunas_texto_applicants = ['cv_pt', 'objetivo_profissional', 'fonte_indicacao', 'titulo_profissional', 'area_atuacao', 'nivel_academico', 'cursos']
colunas_datas_applicants = ['data_criacao','data_atualizacao','data_nascimento']

for col in colunas_texto_applicants:
    df_applicants[col] = df_applicants[col].apply(limpar_texto)

for col in colunas_datas_applicants:
    df_applicants[col] = df_applicants[col].apply(limpar_datas)

df_applicants['remuneracao'] = df_applicants['remuneracao'].apply(limpar_numeros_strings)
df_applicants['ano_conclusao'] = df_applicants['ano_conclusao'].apply(limpar_anos)

## --- TRATANDO COLUNAS TABELA VAGAS ---

colunas_texto_vagas = ['titulo_vaga', 'tipo_contratacao', 'nivel_academico', 'areas_atuacao', 'principais_atividades', 
                       'competencia_tecnicas_e_comportamentais', 'demais_observacoes', 'equipamentos_necessarios', 
                       'habilidades_comportamentais_necessarias']
colunas_datas_vagas = ['limite_esperado_para_contratacao', 'data_inicial', 'data_final']

for col in colunas_texto_vagas:
    df_vagas[col] = df_vagas[col].apply(limpar_texto)

for col in colunas_datas_vagas:
    df_vagas[col] = df_vagas[col].apply(limpar_datas)

## --- TRATANDO COLUNAS TABELA PROSPECTS ---

colunas_texto_prospects = ['titulo', 'situacao_candidado', 'comentario']

for col in colunas_texto_prospects:
    df_prospects[col] = df_prospects[col].apply(limpar_texto)

In [ ]:
# from ydata_profiling import ProfileReport

# ProfileReport(df_applicants).to_file("df_applicants_report.html")
# ProfileReport(df_vagas).to_file("df_vagas_report.html")
# ProfileReport(df_prospects).to_file("df_prospects_report.html")

# from ydata_profiling import ProfileReport

# ProfileReport(df_applicants).to_file("df_applicants_report_2.html")
# ProfileReport(df_vagas).to_file("df_vagas_report_2.html")
# ProfileReport(df_prospects).to_file("df_prospects_report_2.html")

In [ ]:
from ydata_profiling import ProfileReport

ProfileReport(df_applicants).to_file("df_applicants_report_3.html")
ProfileReport(df_vagas).to_file("df_vagas_report_3.html")
ProfileReport(df_prospects).to_file("df_prospects_report_3.html")

In [23]:
situacao_candidado_nao_aprovado = ['nao aprovado rh', 'nao aprovado cliente', 'nao aprovado requisitante', 'recusado', ]

situacao_candidado_aprovado = ['contratado decision', 'prospect', 'entrevista tecnica', 'proposta aceita', 'contratado hunting', 'entrevista cliente', 
                                    'documentacao clt','documentacao pj', 'documentacao cooperado', 'encaminhar proposta']

situacao_candidado_desistente = ['desistiu', 'desistiu contratacao', 'avaliacao rh']

situacao_candidato_inicial = ['encaminhado requisitante', 'interesse nesta vaga', 'inscrito']

df_prospects['situacao_candidado'] = df_prospects['situacao_candidado'].replace(situacao_candidado_nao_aprovado, 'nao_aprovado')
df_prospects['situacao_candidado'] = df_prospects['situacao_candidado'].replace(situacao_candidado_aprovado, 'aprovado')
df_prospects['situacao_candidado'] = df_prospects['situacao_candidado'].replace(situacao_candidado_desistente, 'desistente')
df_prospects['situacao_candidado'] = df_prospects['situacao_candidado'].replace(situacao_candidato_inicial, 'inicial')
df_prospects['situacao_candidado'].value_counts()

situacao_candidado
aprovado        24274
inicial         20678
nao_aprovado     6024
desistente       2783
Name: count, dtype: int64

In [24]:
df_prospects

,cod_vaga,titulo,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,consultor control m,José Vieira,25632,inicial,25-03-2021,25-03-2021,encaminhado pj r 7200 hora,Ana Lívia Moreira
1,4530,consultor control m,Srta. Isabela Cavalcante,25529,inicial,22-03-2021,23-03-2021,encaminhado r 600000 clt full nao empresa aberta,Ana Lívia Moreira
2,4531,20212607395peoplesoft application enginedomain...,Sra. Yasmin Fernandes,25364,aprovado,17-03-2021,12-04-2021,data inicio 12 04 2021,Juliana Cassiano
3,4531,20212607395peoplesoft application enginedomain...,Alexia Barbosa,25360,inicial,17-03-2021,17-03-2021,None,Juliana Cassiano
4,4532,None,NaN,NaN,None,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...
56697,14219,None,NaN,NaN,None,NaN,NaN,None,NaN
56698,14220,consultor senior especialista sap lestra 1433,Ana Cardoso,16828,desistente,26-02-2025,28-02-2025,recebeu confirmacao outro processo seletivo re...,Elisa Nunes
56699,14220,consultor senior especialista sap lestra 1433,Pedro Lucas das Neves,15042,inicial,28-02-2025,28-02-2025,None,Elisa Nunes
56700,14221,consultor senior oracle epm fccs 1434,Maria Eduarda Cassiano,49190,aprovado,26-02-2025,26-02-2025,None,Luna Correia


In [25]:
df_vagas

,cod_vaga,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,cliente,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,tipo_contratacao,...,nivel_ingles,nivel_espanhol,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1
0,5185,04-05-2021,NaT,operation lead,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,clt full,...,Avançado,Fluente,ti sistemas ferramentas,operations lead roles responsibilities the ope...,required skills prior experience in cloud infr...,100 remoto periodo 5 6 meses,nenhum,None,-,R$
1,5184,04-05-2021,NaT,consultor pp qm senior,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,clt full,...,Fluente,Nenhum,ti desenvolvimento programacao,consultor pp qm sr consultor pp qm senior expe...,consultor pp qm senior experiencia projetos ro...,inicio imediato fim jan 22,nenhum,None,-,R$
2,5183,04-05-2021,NaT,analista pl jr c sql,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,clt full,...,Nenhum,Intermediário,ti sistemas ferramentas,descricao atividades monitoramento interfaces ...,requisitos mandatorios conhecimentos tecnicos ...,localizacao remoto perfil analista pleno jr pe...,nenhum,None,-,R$
3,5182,04-05-2021,2021-05-18,technical architect 11894809,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,pj autonomo clt full,...,Básico,Básico,ti projetos,descricao comentario architecture frameworks r...,descricao comentario architecture frameworks r...,budgeted rate indicate currency and type hourl...,notebook padrao,None,- p/ mês (168h),fechado
4,5181,04-05-2021,NaT,consultor sap authorization bca pleno senior,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,clt full,...,Intermediário,Nenhum,ti sap,experiencia consultor sap authorization bca pl...,experiencia consultor sap authorization bca pl...,contratacao clt full decision locacao remota s...,nenhum,None,-,R$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14076,12368,28-02-2024,2024-03-31,pmp 5203658 dynamics,Miller-Curry,Guilherme Campos,Decision Campinas,Sra. Juliana Oliveira,Sra. Joana Sousa,clt full,...,Nenhum,<NA>,ti desenvolvimento programacao,dynamics,dynamics,None,None,None,"123,00 -",APPLICATION DEVELOPER-MICROSOFT ANALYTICS - ba...
14077,12367,28-02-2024,2024-02-29,mm im fsr 3910,Bishop-Reed,Lívia Vasconcelos,Decision São Paulo,Manuella Jesus,Srta. Bella Ferreira,clt full pj autonomo,...,Avançado,Básico,ti projetos,1 deve ter boa experiencia configuracao mmim 2...,ingles avancado comunicar global interacao ind...,fsr 3910,None,None,168 -,Fechado
14078,12366,27-02-2024,2024-02-29,964 assistente fiscal financeiro,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Maria Clara Pires,clt full pj autonomo,...,Básico,Básico,ti projetos,qualificacoes tecnicas conhecimento basico ent...,local alphaville barueri regiao sp idioma port...,modelo trabalho hibrido 3 vezes semana escrito...,None,None,168 -,Fechado
14079,12365,27-02-2024,2024-02-29,966 analista pleno senior area fiscal,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Melina Montenegro,clt full pj autonomo,...,Básico,Básico,ti projetos,qualificacoes tecnicas apuracao impostos retid...,local alphaville barueri sp regiao grande sao ...,modelo trabalho hibrido 3 vezes semana escrito...,None,None,168 -,Fechado


In [26]:
df_applicants

,cod_applicant,cv_pt,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,sabendo_de_nos_por,data_atualizacao,...,endereco,download_cv,titulo_profissional,area_atuacao,remuneracao,nivel_academico,nivel_ingles,nivel_espanhol,cursos,ano_conclusao
0,31000,assistente administrativo santosbatista itapec...,(11) 97048-2708,None,2021-10-11 07:29:49,Luna Correia,carolina_aparecida@gmail.com,<NA>,<NA>,2021-10-11 07:29:49,...,<NA>,NaN,None,None,<NA>,None,<NA>,<NA>,None,NaN
1,31001,formacao academica ensino medio 2o grau ensino...,(11) 93723-4396,analista administrativo,2021-10-11 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,2021-11-11 11:10:31,...,são paulo,NaN,analista administrativo,administrativa,19,ensino superior incompleto,Nenhum,Nenhum,None,NaN
2,31002,objetivo area administrativa financeira resumo...,(11) 92399-9824,administrativo financeiro,2021-10-11 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,2021-10-11 11:42:36,...,são paulo,NaN,administrativo financeiro,administrativa,"2.5,",ensino superior completo,Intermediário,Básico,administracao empresas,2012.0
3,31003,formacao ensino medio completo informatica int...,(11) 98100-1727,area administrativa,2021-10-11 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,2021-10-11 16:04:51,...,são paulo,NaN,area administrativa,administrativa,"11,",ensino superior incompleto,Nenhum,Nenhum,None,NaN
4,31004,ultima atualizacao 09 11 2021 sp ensino superi...,(11) 92517-2678,None,2021-10-11 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,<NA>,<NA>,2021-10-11 09:22:03,...,<NA>,NaN,None,None,<NA>,None,<NA>,<NA>,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42477,5995,None,(11) 92622-2088,crm lider projetos,2018-03-14 16:35:14,pelo próprio candidato,gabrielly_novaes@gmail.com,<NA>,Outros,2018-03-14 16:35:14,...,<NA>,NaN,crm lider projetos,None,,None,<NA>,<NA>,None,NaN
42478,5996,None,(41) 93009-5141,crm,2018-03-14 16:38:28,pelo próprio candidato,sr._gabriel_rios@gmail.com,<NA>,Outros,2018-03-14 16:38:28,...,<NA>,NaN,crm,None,,None,<NA>,<NA>,None,NaN
42479,5997,None,(11) 97837-3941,crm,2018-03-14 16:41:52,pelo próprio candidato,dr._mathias_da_cunha@gmail.com,<NA>,Outros,2018-03-14 16:41:52,...,<NA>,NaN,crm,None,,None,<NA>,<NA>,None,NaN
42480,5998,None,(61) 96806-8075,crm,2018-03-14 16:42:49,pelo próprio candidato,bruna_vieira@hotmail.com,<NA>,Outros,2018-03-14 16:42:49,...,<NA>,NaN,crm,None,,None,<NA>,<NA>,None,NaN


In [41]:
df_prospects_and_vagas = df_prospects.merge(df_vagas, how='inner', on='cod_vaga')#.merge(df_applicants, how='left', left_on='codigo', right_on='cod_applicant')

# for situacao in df_prospects['situacao_candidado'].unique():
#     print(f"Situação: {situacao}")
#     print(df_prospects[df_prospects['situacao_candidado'] == situacao]['cod_vaga'].value_counts())

In [ ]:
ProfileReport(df_prospects_and_vagas).to_file("df_prospects_and_vagas_report.html")

In [43]:
df_prospects_and_vagas

,cod_vaga,titulo,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,data_requicisao,...,nivel_ingles,nivel_espanhol,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1
0,4530,consultor control m,José Vieira,25632,inicial,25-03-2021,25-03-2021,encaminhado pj r 7200 hora,Ana Lívia Moreira,10-03-2021,...,Nenhum,Nenhum,ti desenvolvimento programacao,experiencia comprovada projetos controlm,experiencia comprovada projetos controlm,contratacao pj projeto pontual 2 3 meses clien...,nenhum,None,-,R$
1,4530,consultor control m,Srta. Isabela Cavalcante,25529,inicial,22-03-2021,23-03-2021,encaminhado r 600000 clt full nao empresa aberta,Ana Lívia Moreira,10-03-2021,...,Nenhum,Nenhum,ti desenvolvimento programacao,experiencia comprovada projetos controlm,experiencia comprovada projetos controlm,contratacao pj projeto pontual 2 3 meses clien...,nenhum,None,-,R$
2,4531,20212607395peoplesoft application enginedomain...,Sra. Yasmin Fernandes,25364,aprovado,17-03-2021,12-04-2021,data inicio 12 04 2021,Juliana Cassiano,10-03-2021,...,Nenhum,Nenhum,gestao alocacao recursos ti,key skills required the job are peoplesoft app...,recurso peoplesoft responsabilidades projetar ...,remoto presencial tempo indeterminado,notebook padrao,None,-,hora
3,4531,20212607395peoplesoft application enginedomain...,Alexia Barbosa,25360,inicial,17-03-2021,17-03-2021,None,Juliana Cassiano,10-03-2021,...,Nenhum,Nenhum,gestao alocacao recursos ti,key skills required the job are peoplesoft app...,recurso peoplesoft responsabilidades projetar ...,remoto presencial tempo indeterminado,notebook padrao,None,-,hora
4,4532,None,NaN,NaN,None,NaN,NaN,None,NaN,11-03-2021,...,Nenhum,Nenhum,gestao alocacao recursos ti,consultor senior experiencia localizacao brasi...,desejavel conhecimento segmento ol gas atendim...,apenas pj clt full,None,tempo projeto 1 ano requested start date110320...,-,Valor Aberto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56533,14219,None,NaN,NaN,None,NaN,NaN,None,NaN,26-02-2025,...,Básico,<NA>,ti projetos,importante idioma ingles espanhol level 9 8 le...,skills aspnet javascript react csharp applicat...,primary contact pfigueiredo wbs z026gl70 indir...,None,None,168 - p/ hora,Fechado
56534,14220,consultor senior especialista sap lestra 1433,Ana Cardoso,16828,desistente,26-02-2025,28-02-2025,recebeu confirmacao outro processo seletivo re...,Elisa Nunes,26-02-2025,...,Intermediário,<NA>,ti sap,vaga consultor senior especialista sap lestra ...,conhecimentos tecnicos requeridos consultor sa...,None,None,None,168 -,Fechado
56535,14220,consultor senior especialista sap lestra 1433,Pedro Lucas das Neves,15042,inicial,28-02-2025,28-02-2025,None,Elisa Nunes,26-02-2025,...,Intermediário,<NA>,ti sap,vaga consultor senior especialista sap lestra ...,conhecimentos tecnicos requeridos consultor sa...,None,None,None,168 -,Fechado
56536,14221,consultor senior oracle epm fccs 1434,Maria Eduarda Cassiano,49190,aprovado,26-02-2025,26-02-2025,None,Luna Correia,26-02-2025,...,Básico,<NA>,ti projetos,vaga consultor senior oracle epm fccs modelo t...,conhecimentos tecnicos requeridos oracle epm f...,None,None,None,168 -,Fechado
